In [1]:
%matplotlib inline
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder, RobustScaler, MinMaxScaler
from sklearn import preprocessing

import matplotlib.image as mpimg
from skimage.io import imread, imshow
from skimage import data, color, io, filters, morphology,transform, exposure, feature, util
from scipy import ndimage
#import Tensorflow namespaces

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions

import pickle

#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
from os import walk, path
from random import shuffle
from skimage.io import imread, imshow
from skimage import data, color, io, filters, morphology,transform, exposure, feature, util
import matplotlib.pyplot as plt
import numpy as np
import glob, random

In [3]:
def load_images(path):
    
    images = []
    for image_file in os.listdir(path):

        image = io.imread(os.path.join(path, image_file))
        if image is not None:
            images.append(image)
            

    return images

vehicles = load_images('./vehicles')
non_vehicles = load_images('./non-vehicles')

In [4]:
Xs = vehicles + non_vehicles
Xs = np.array(Xs)

Xs = Xs.astype('float32')
Xs /= 255

In [5]:
# Generate labels 1 for vehicles and 0 for non-vehicles
class_labels = np.array([1] * len(vehicles) + [0] * len(non_vehicles))

In [12]:
# Devide the Xs into training and test sets
X_train, X_test, y_train, y_test = train_test_split(Xs, class_labels, test_size=0.2, random_state=20)

y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(14208, 64, 64, 3)
(3552, 64, 64, 3)
(14208, 2)
(3552,)


In [14]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)

# Neural network parameters
#-----------------------------------------------
#-----------------------------------------------
batch_size = 64 #
epochs = 50 # 

kernel_size=(3,3)
#-----------------------------------------------
#-----------------------------------------------
num_classes = 2
input_shape = ( X_train.shape[1], X_train.shape[2], X_train.shape[3])


# Model
model = Sequential()
model.add(Conv2D(16, kernel_size=kernel_size, input_shape=input_shape)) 
model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(16, kernel_size=kernel_size)) 

model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))


model.add(Flatten()) 
model.add(Dropout(0.4)) 
model.add(Dense(20, activation='relu')) 

model.add(Dropout(0.4))

model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [15]:
# Trainen van het CNN
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, min_delta=0.001, patience=3, restore_best_weights=True)

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1, callbacks=[es])


Epoch 1/50
178/178 [==============================] - 3s 12ms/step - loss: 0.7125 - accuracy: 0.6846 - val_loss: 0.7196 - val_accuracy: 0.5028
Epoch 2/50
178/178 [==============================] - 2s 10ms/step - loss: 0.4484 - accuracy: 0.8012 - val_loss: 0.6060 - val_accuracy: 0.6562
Epoch 3/50
178/178 [==============================] - 2s 10ms/step - loss: 0.3615 - accuracy: 0.8374 - val_loss: 0.3353 - val_accuracy: 0.8441
Epoch 4/50
178/178 [==============================] - 2s 11ms/step - loss: 0.3066 - accuracy: 0.8674 - val_loss: 0.2354 - val_accuracy: 0.9032
Epoch 5/50
178/178 [==============================] - 2s 11ms/step - loss: 0.2755 - accuracy: 0.8850 - val_loss: 0.2057 - val_accuracy: 0.9152
Epoch 6/50
178/178 [==============================] - 3s 18ms/step - loss: 0.2469 - accuracy: 0.8971 - val_loss: 0.1762 - val_accuracy: 0.9328
Epoch 7/50
178/178 [==============================] - 2s 11ms/step - loss: 0.2178 - accuracy: 0.9111 - val_loss: 0.1545 - val_accuracy: 0.9430

In [16]:
y_predicted = np.argmax(model.predict(X_test),axis=-1)

print(classification_report(y_test, y_predicted))

cf = confusion_matrix(y_test, y_predicted)

print(cf)
print(accuracy_score(y_test, y_predicted) * 100) 

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1800
           1       1.00      0.99      0.99      1752

    accuracy                           0.99      3552
   macro avg       0.99      0.99      0.99      3552
weighted avg       0.99      0.99      0.99      3552

[[1794    6]
 [  13 1739]]
99.46509009009009
